## Periodic 2-D grid of Ququarts

Each node has 4 links to its four nearest neighbours, hence a ququad at each site. X axis points in the horizontal direction, Y in the vertical. 

Coin: Grover or Fourier

Phase randomization: optional

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
########################################## parameters
N =  729
Nx = np.sqrt(N).astype(int)
Ny = np.sqrt(N).astype(int)

W =  [0.0] #np.linspace(0,0.5,10)

grover = np.ones((4,4))
for i in range(0,4):
    grover[i,i] = grover[i,i]*(-1)
grover /= 2
#print(grover)
fourier = np.ones((4,4), dtype=np.complex128)
for i in range(1,4):
    for j in range(1,4):
        fourier[i][j] = np.exp(2j*np.pi*i*j/4)
fourier /= np.sqrt(4)
#print(fourier)
########################################## functions ##
    
def S(state):
    for j in range(0,Ny):
        for i in range(0,Nx):
            state[i%Nx,j%Ny,1], state[(i+1)%Nx,j%Ny,3] = state[(i+1)%Nx,j%Ny,3], state[i%Nx,j%Ny,1]
            state[i%Nx,j%Ny,0], state[i%Nx,(j+1)%Ny,2] = state[i%Nx,(j+1)%Ny,2], state[i%Nx,j%Ny,0]
    # for j in range(0,Ny-1):
    #     for i in range(0,Nx-1):
    #         state[i,j,1], state[i+1,j,3] = state[i+1,j,3], state[i,j,1]
    #         state[i,j,0], state[i,j+1,2] = state[i,j+1,2], state[i,j,0]
    # for i in range(0,Ny):
    #     state[Nx-1,i,1], state[0,i,3] = state[0,i,3], state[Nx-1,i,1]
    # for i in range(0,Nx):
    #     state[i,Ny-1,0], state[i,0,2] = state[i,0,2], state[i,Ny-1,0]
    # #print("state after S", state)
    return state


def C_Grover(state):
    for i in range(0,Nx):
        for j in range(0,Ny):
            state[i,j,0:4] = np.matmul(grover,state[i,j,0:4])
    #print("state after C", state)
    return state

def C_Fourier(state):
    for i in range(0,Nx):
        for j in range(0,Ny):
            state[i,j,0:4] = np.matmul(fourier,state[i,j,0:4])
    #print("state after C", state)
    return state

def phase(state,w):
    dim = (state.shape[0], state.shape[1])
    #print(dim)
    for i in range(0,4):
        rnd = np.random.uniform(-w,w,size=dim)
        state[:,:,i] = state[:,:,i]*np.exp(2*np.pi*1j*rnd)
    return state

# to check normalization, plus to get amplitudes of the state at various position n
def magn(state):
    magnit = np.zeros((Nx,Ny))
    #check = 0
    for i in range(0,Nx):
        for j in range(0,Ny):
            magnit[i,j] = (state[i,j,0].conjugate()*state[i,j,0] + state[i,j,1].conjugate()*state[i,j,1] 
                           + state[i,j,2].conjugate()*state[i,j,2] + state[i,j,3].conjugate()*state[i,j,3])
            #check += magnit[i,j]
    #print("check value", check)
    return magnit

In [20]:
def main(w, walkers=1):
    # state declaration and initialization
    state = np.zeros((Nx,Ny,4), dtype=np.complex128)  
    ## state initialized at the middle point of the lattice
    # state[int(Nx/2)][int(Ny/2)][0:4] = 0.5*np.sqrt(walkers)
    ## uncomment the following to initialize the state at a random position
    I = np.random.randint(0,Nx)
    J = np.random.randint(0,Ny)
    print("Initial position: ", I, J)
    state[I,J,0:4] = np.sqrt(walkers)/2
    # uncomment the following to have a uniform superposition instead
    # state[:,:,0:4].fill(np.sqrt(walkers/4*N))   
    
    # custom runtime
    runtime = N*10
    # uncomment the following to have runtime in accordance with the lattice size
    # runtime = int(Nx*np.log(Nx))
    
    # magnitude stores wave amplitude at each position for all timestamps
    magnitude = np.zeros((Nx,Ny,runtime))
    magnitude[:,:,0] = magn(state)
    
    # run the walker
    for j in range(1,runtime):
        # state = S(C_Grover(phase(state,w)))
        # state = S(C_Fourier(phase(state,w)))
        #state = S(C_Grover(state))
        state = S(C_Fourier(state))
        magnitude[:,:,j] = magn(state)
    return magnitude, runtime

In [21]:
def get_threshold_byavg(magnit):
    Pth = np.zeros((Nx,Ny))
    L = len(magnit[0,0,0,:])
    for j in range(0,Ny):
        for i in range(0,Nx):
            Pth[i,j] = 2*np.mean(np.sort(magnit[i,j,:])[int(0.666*L):])
    return Pth
    
def get_threshold_bydev(magnit):
    Pth = np.zeros((Nx,Ny))
    for j in range(0,Ny):
        for i in range(0,Nx):
            Pth[i,j] = np.mean(magnit[i,j,:]) + 3*np.std(magnit[i,j,:])
    return Pth

def EE_analysis(magnitude, means,stds,runtime,stab_time,w,run):
    count = 0
    P_th = means + 3*stds
    # print(P_th.shape)
    for j in range(0,Ny):
        for i in range(0,Nx):
            for time in range(stab_time,runtime):
                if (magnitude[i,j,time] > P_th[i,j]):
                    count += 1
    perc = 100*count/(N*(runtime-stab_time))
    print("On run = {}, rouge event count for w = {} is: ".format(run,w), count,\
          " out of total runtime*N events, that is {} %".format(perc))
    return perc

In [22]:
k = 0 ## k is the index for N, we work for now with one value of N only. To make use of more values of N, we need to loop over k later on.
RUN = 10
means_run = np.zeros((Nx, Ny, RUN))
stds_run = np.zeros((Nx, Ny, RUN))
stab_time = N

perc_ee = np.zeros(RUN)
w = W[0]
for run in range(0,RUN):
    magnitude, runtime = main(w)
    for j in range(0,Ny):
        for i in range(0,Nx):
            means_run[i,j, run] = np.mean(magnitude[i,j,stab_time:])
            stds_run[i,j, run] = np.std(magnitude[i,j,stab_time:])
    perc_ee[run] = EE_analysis(magnitude, means_run[:,:,run],stds_run[:,:,run],runtime,stab_time,w,run)
means = np.mean(means_run, axis=2)
stds = np.mean(stds_run, axis=2)
print("Average percentage of rouge events over {} runs is: ".format(RUN), np.mean(perc_ee), " %")
print("Standard deviation of percentage of rouge events over {} runs is: ".format(RUN), np.std(perc_ee), " %")

Initial position:  21 0


/var/folders/4g/hvkx9ls12r741x0652yfb8_40000gn/T/ipykernel_9627/1850063057.py:69: ComplexWarning: Casting complex values to real discards the imaginary part
  magnit[i,j] = (state[i,j,0].conjugate()*state[i,j,0] + state[i,j,1].conjugate()*state[i,j,1]


On run = 0, rouge event count for w = 0.0 is:  64749  out of total runtime*N events, that is 1.3537407413679663 %
Initial position:  20 4
On run = 1, rouge event count for w = 0.0 is:  64749  out of total runtime*N events, that is 1.3537407413679663 %
Initial position:  23 23
On run = 2, rouge event count for w = 0.0 is:  64749  out of total runtime*N events, that is 1.3537407413679663 %
Initial position:  2 15
On run = 3, rouge event count for w = 0.0 is:  64749  out of total runtime*N events, that is 1.3537407413679663 %
Initial position:  8 4
On run = 4, rouge event count for w = 0.0 is:  64749  out of total runtime*N events, that is 1.3537407413679663 %
Initial position:  16 3
On run = 5, rouge event count for w = 0.0 is:  64749  out of total runtime*N events, that is 1.3537407413679663 %
Initial position:  15 12
On run = 6, rouge event count for w = 0.0 is:  64749  out of total runtime*N events, that is 1.3537407413679663 %
Initial position:  22 14
On run = 7, rouge event count fo

In [23]:
print(np.shape(means))
print(np.shape(stds))

expected_std = np.sqrt( (1/N) * (1 - 1/N) )
expected_mean = 1/N

print("Mean of distributions, averaged over {} nodes is: ".format(N), np.mean(means))
print("expected mean of the distributions is: ", expected_mean)
print("the standard deviation of the obtained means is: ", np.std(means))
print("standard deviatio of the probability amplitude is: ", np.mean(stds))
print("expected standard deviation of the distributions is: ", expected_std)
# print("mean squared error: ", mse)

(27, 27)
(27, 27)
Mean of distributions, averaged over 729 nodes is:  0.0013717421124828531
expected mean of the distributions is:  0.0013717421124828531
the standard deviation of the obtained means is:  0.0001043969124638962
standard deviatio of the probability amplitude is:  0.00077886508770485
expected standard deviation of the distributions is:  0.0370116256878794
